### ***7***

# Multiclass Logistic Regression + BGD + 5-Fold

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%reset-f

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [7]:
path = '/content/drive/MyDrive/ML/data_q6_q7.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)
insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)

In [8]:
# hypothesis for logistic regression which is the prediction 
def hypothesis(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost(h,insts,m):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  return c
# update of weight values for logistic regression
def update(w,alpha,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] = w[i] - alpha*d[i]

  return w
def update_stotastic(w,alpha,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = w[i] - alpha*d[i]

  return w

def performance(mat,m):
  p = []
  for i in range(3):
    ia = mat[i,i]/(mat[i,0]+mat[i,1]+mat[i,2])
    p.append(ia)
  
  p.append((mat[0,0]+mat[1,1]+mat[2,2])/m)
  return p


In [ ]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.0001,0.001,100)
    b_min = 10000
    c = np.ones(len(alpha))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    for j in range(len(alpha)):
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for i in range(itr):
        h = hypothesis(w,train,m_tr,n)
        w = update(w,alpha[j],train,h,m_tr,n)
      h = hypothesis(w,train,m_tr,n)
      c[j] = cost(h,train,m_tr)
      if c[j] < b_min:
        min = j
        b_min = c[j]
        w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,m_tr)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


accuracy_class1: 0.7319298245614035 
accuracy_class2: 0.9511111111111111 
accuracy_class3: 0.9549450549450551 
overall_accuracy: 0.870267131242741


# Multiclass Logistic Regression + MBG + 5-Fold

In [ ]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.001,0.01,100)
    b_min = 10000
    c = np.ones(len(alpha))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    m_batch =20
    for j in range(len(alpha)):
      w = np.random.rand(n)
      for i in range(itr):
        np.random.shuffle(train)
        h = hypothesis(w,train,m_tr,n)
        w = update(w,alpha[j],train,h,m_batch,n)
      h = hypothesis(w,train,m_tr,n)
      c[j] = cost(h,train,m_tr)
      if c[j] < b_min:
        min = j
        b_min = c[j]
        w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,m_tr)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


accuracy_class1: 0.8276942355889723 
accuracy_class2: 0.9511111111111111 
accuracy_class3: 0.8673561732385263 
overall_accuracy: 0.8702671312427409


# Multiclass Logistic Regression + SGD + 5-Fold

In [ ]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts_s[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_s,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.01,0.1,100)
    b_min = 10000
    c = np.ones(len(alpha))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    for j in range(len(alpha)):
      w = np.random.rand(n)
      for i in range(itr):
        index = np.random.randint(m_tr)
        h = hypothesis(w,train,m_tr,n)
        w = update_stotastic(w,alpha[j],train,h,index,n)
      h = hypothesis(w,train,m_tr,n)
      c[j] = cost(h,train,m_tr)
      if c[j] < b_min:
        min = j
        b_min = c[j]
        w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,m_tr)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


accuracy_class1: 0.7190225563909776 
accuracy_class2: 0.9622222222222222 
accuracy_class3: 0.9549450549450549 
overall_accuracy: 0.8702671312427409


# Multiclass Logistic Regression + BGD + L2-NORM + 5-Fold

In [10]:
%reset-f

In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy
import warnings

In [12]:
path = '/content/drive/MyDrive/ML/data_q6_q7.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)
insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)

In [13]:
# hypothesis for logistic regression which is the prediction 
def hypothesis(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost(h,insts,lamb,m,w):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  return c + 0.5*lamb*(np.dot(w,w))
# update of weight values for logistic regression
def update(w,alpha,lamb,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] = w[i]*(1-alpha*lamb) - alpha*d[i]

  return w
def update_stotastic(w,alpha,lamb,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = w[i]*(1-alpha*lamb) - alpha*d[i]

  return w

def performance(mat,m):
  p = []
  for i in range(3):
    ia = mat[i,i]/(mat[i,0]+mat[i,1]+mat[i,2])
    p.append(ia)
  
  p.append((mat[0,0]+mat[1,1]+mat[2,2])/m)
  return p

In [ ]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.0001,0.001,10)
    lamb = np.linspace(0,100,10)
    b_min = 10000
    c = np.ones(len(lamb))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    for j in range(len(lamb)):
      for a in alpha:
        w = np.random.rand(n)
        j_list = np.ones(itr)
        for i in range(itr):
          h = hypothesis(w,train,m_tr,n)
          w = update(w,a,lamb[j],train,h,m_tr,n)
        h = hypothesis(w,train,m_tr,n)
        c[j] = cost(h,train,lamb[j],m_tr,w)
        if c[j] < b_min:
          min = j
          b_min = c[j]
          w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,lamb[min],m_tr,w_opt)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)



accuracy_class1: 0.7746430040547688 
accuracy_class2: 0.9221848739495797 
accuracy_class3: 0.9761403508771929 
overall_accuracy: 0.8852497096399536


# Multiclass Logistic Regression + MBG + L2-NORM + 5-Fold

In [ ]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.001,0.01,10)
    lamb = np.linspace(0,100,10)
    b_min = 10000
    c = np.ones(len(lamb))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    m_batch = 20
    for j in range(len(lamb)):
      for a in alpha:
        w = np.random.rand(n)
        j_list = np.ones(itr)
        for i in range(itr):
          np.random.shuffle(train)
          h = hypothesis(w,train,m_tr,n)
          w = update(w,a,lamb[j],train,h,m_batch,n)
        h = hypothesis(w,train,m_tr,n)
        c[j] = cost(h,train,lamb[j],m_tr,w)
        if c[j] < b_min:
          min = j
          b_min = c[j]
          w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,lamb[min],m_tr,w_opt)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  print(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


[[13  1  0]
 [ 0  9  0]
 [ 5  0 14]]
[[12  1  1]
 [ 0 15  0]
 [ 1  0 12]]
[[16  0  1]
 [ 6 11  0]
 [ 2  0  6]]
[[11  2  0]
 [ 0 14  0]
 [ 9  0  6]]
[[11  0  0]
 [ 2 13  0]
 [10  0  5]]
accuracy_class1: 0.9146089204912734 
accuracy_class2: 0.9027450980392157 
accuracy_class3: 0.6286504723346829 
overall_accuracy: 0.8033681765389081


# Multiclass Logistic Regression + SGD + L2-NORM + 5-Fold

In [15]:
#warnings.filterwarnings('ignore')
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.01,0.1,10)
    lamb = np.linspace(0,50,10)
    b_min = 10000
    c = np.ones(len(lamb))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    for j in range(len(lamb)):
      for a in alpha:
        w = np.random.rand(n)
        j_list = np.ones(itr)
        for i in range(itr):
          index = np.random.randint(m_tr)
          h = hypothesis(w,train,m_tr,n)
          w = update_stotastic(w,a,lamb[j],train,h,index,n)
        h = hypothesis(w,train,m_tr,n)
        c[j] = cost(h,train,lamb[j],m_tr,w)
        if c[j] < b_min:
          min = j
          b_min = c[j]
          w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,lamb[min],m_tr,w_opt)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

  
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init

accuracy_class1: 0.6252350427350427 
accuracy_class2: 0.5872727272727273 
accuracy_class3: 0.9833333333333334 
overall_accuracy: 0.7369337979094077


# Multiclass Logistic Regression + BGD + L1-NORM + 5-Fold

In [ ]:
%reset-f

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy
import warnings

In [ ]:
path = '/content/drive/MyDrive/ML/data_q6_q7.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)
insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)

In [ ]:
# hypothesis for logistic regression which is the prediction 
def hypothesis(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost(h,insts,lamb,m,w):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  w_abs = np.absolute(w)
  w_abs = np.sum(w_abs)
  return c/m + 0.5*lamb*(w_abs)
# update of weight values for logistic regression
def update(w,alpha,lamb,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] = (w[i]-alpha*lamb*np.sign(w[i])) - alpha*d[i]

  return w
def update_stotastic(w,alpha,lamb,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = (w[i]-alpha*lamb*np.sign(w[i])) - alpha*d[i]

  return w

def performance(mat,m):
  p = []
  for i in range(3):
    ia = mat[i,i]/(mat[i,0]+mat[i,1]+mat[i,2])
    p.append(ia)
  
  p.append((mat[0,0]+mat[1,1]+mat[2,2])/m)
  return p


In [16]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.0001,0.001,10)
    lamb = np.linspace(0,100,10)
    b_min = 10000
    c = np.ones(len(lamb))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    for j in range(len(lamb)):
      for a in alpha:
        w = np.random.rand(n)
        j_list = np.ones(itr)
        for i in range(itr):
          h = hypothesis(w,train,m_tr,n)
          w = update(w,a,lamb[j],train,h,m_tr,n)
        h = hypothesis(w,train,m_tr,n)
        c[j] = cost(h,train,lamb[j],m_tr,w)
        if c[j] < b_min:
          min = j
          b_min = c[j]
          w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,lamb[min],m_tr,w_opt)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)



accuracy_class1: 0.6931837606837608 
accuracy_class2: 0.9566666666666667 
accuracy_class3: 0.9690476190476192 
overall_accuracy: 0.875609756097561


# Multiclass Logistic Regression + MBG + L1-NORM + 5-Fold

In [17]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.001,0.01,10)
    lamb = np.linspace(0,100,10)
    b_min = 10000
    c = np.ones(len(lamb))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    m_batch = 20
    for j in range(len(lamb)):
      for a in alpha:
        w = np.random.rand(n)
        j_list = np.ones(itr)
        for i in range(itr):
          np.random.shuffle(train)
          h = hypothesis(w,train,m_tr,n)
          w = update(w,a,lamb[j],train,h,m_batch,n)
        h = hypothesis(w,train,m_tr,n)
        c[j] = cost(h,train,lamb[j],m_tr,w)
        if c[j] < b_min:
          min = j
          b_min = c[j]
          w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,lamb[min],m_tr,w_opt)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

    
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  print(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


[[ 7  1  2]
 [ 1 14  0]
 [ 3  0 14]]
[[12  0  0]
 [ 1 15  0]
 [ 3  0 11]]
[[16  0  0]
 [ 1 10  0]
 [ 2  0 13]]
[[15  1  2]
 [ 0 12  0]
 [ 4  0  8]]
[[11  0  2]
 [ 6 10  0]
 [ 1  0 11]]
accuracy_class1: 0.875897435897436 
accuracy_class2: 0.8809848484848484 
accuracy_class3: 0.8118487394957983 
overall_accuracy: 0.8560975609756099


# Multiclass Logistic Regression + SGD + L1-NORM + 5-Fold

In [21]:
warnings.filterwarnings('ignore')
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  opt_weights = [] # stores the weight values for all the models
  opt_cost = [] # stores the final optimal cost value at the end of each group
  for l in range(1,4):
    train = copy.deepcopy(insts_tr)
    train[:,-1] = (train[:,-1]==l).astype(int) # assignes the value 1 if the level is i
    m_tr = len(insts_tr[:,0])
    itr = 500
    alpha = np.linspace(0.01,0.1,10)
    lamb = np.linspace(0,50,10)
    b_min = 10000
    c = np.ones(len(lamb))
    w_opt = np.zeros(n)  #stores the optimal weight value
    min=0
    for j in range(len(lamb)):
      for a in alpha:
        w = np.random.rand(n)
        j_list = np.ones(itr)
        for i in range(itr):
          index = np.random.randint(m_tr)
          h = hypothesis(w,train,m_tr,n)
          w = update_stotastic(w,a,lamb[j],train,h,index,n)
        h = hypothesis(w,train,m_tr,n)
        c[j] = cost(h,train,lamb[j],m_tr,w)
        if c[j] < b_min:
          min = j
          b_min = c[j]
          w_opt = w
    h = hypothesis(w_opt,train,m_tr,n)
    opt_cost.append(cost(h,train,lamb[min],m_tr,w_opt)) # cal the final cost value
    opt_weights.append(w_opt) # append the optimal weight values for every group  
  m_te = len(insts_te[:,0]) 
  y_acu = insts_te[:,-1].astype(int)
  y_pre = np.zeros(len(insts_te)).astype(int)
  for i in range(len(opt_weights)):
    h_te = hypothesis(opt_weights[i],insts_te,m_te,n)
    y_pe = np.round(h_te)
    
    for j in range(len(y_pe)):
      if y_pe[j] == 1 and y_pre[j] == 0:
        y_pre[j] = i+1
      if y_pe[j] == 1 and y_pre[j] !=0:
        if opt_cost[i] <= opt_cost[y_pre[j]-1]:
          y_pre[j] = i+1
      if y_pre[j] == 0 and i==2:
        y_pre[j] = 1

  
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pre, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  p = performance(confmat,m_te)
  accuracy_class1 += p[0]
  accuracy_class2 += p[1]
  accuracy_class3 += p[2]
  overall_accuracy += p[3]

print('accuracy_class1:',accuracy_class1/K,'\naccuracy_class2:',accuracy_class2/K,'\naccuracy_class3:',accuracy_class3/K,'\noverall_accuracy:',overall_accuracy/K)


accuracy_class1: 0.5414102564102564 
accuracy_class2: 0.7093939393939394 
accuracy_class3: 0.9857142857142858 
overall_accuracy: 0.7559814169570267
